# libs init

In [1]:
import re
import itertools
import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import AllChem

# Чтение данных и EDA

In [2]:
df = pd.read_csv('../../data/target_cox2_IC50_full_features.csv', delimiter = ';')
df

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties,Action Type,Standard Text Value,Value
0,CHEMBL70104,NaN,NaN,364.46,0.0,3.49,3,CC1(C)OC(=O)C(OC2CCCCC2)=C1c1ccc(S(C)(=O)=O)cc1,IC50,'=',...,CHEMBL1131821,1,Scientific Literature,Bioorg Med Chem Lett,1999.0,NaN,NaN,NaN,NaN,0.04
1,CHEMBL96098,NaN,NaN,372.88,0.0,4.43,34,CCc1ccc(-c2ncc(Cl)cc2-c2ccc(S(C)(=O)=O)cc2)cn1,IC50,'=',...,CHEMBL1130609,1,Scientific Literature,Bioorg Med Chem Lett,1998.0,NaN,NaN,NaN,NaN,1.70
2,CHEMBL97179,NaN,NaN,413.90,1.0,5.19,5,CCCCOC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc(OC)...,IC50,'=',...,CHEMBL1133457,1,Scientific Literature,J Med Chem,2000.0,NaN,NaN,NaN,NaN,0.05
3,CHEMBL6,INDOMETHACIN,4.0,357.79,0.0,3.93,Indomethacin,COc1ccc2c(c1)c(CC(=O)O)c(C)n2C(=O)c1ccc(Cl)cc1,IC50,'<',...,CHEMBL1131754,1,Scientific Literature,Bioorg Med Chem Lett,1999.0,NaN,NaN,NaN,NaN,200.00
4,CHEMBL355781,NaN,NaN,390.46,0.0,4.25,21,CS(=O)(=O)c1ccc(-c2csc(CC(=O)O)c2-c2ccc(F)cc2)cc1,IC50,'>',...,CHEMBL1128908,1,Scientific Literature,Bioorg Med Chem Lett,1996.0,CHEMBL3308860,NaN,NaN,NaN,10.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7974,CHEMBL118,CELECOXIB,4.0,381.38,0.0,3.51,Celecoxib,Cc1ccc(-c2cc(C(F)(F)F)nn2-c2ccc(S(N)(=O)=O)cc2...,IC50,'=',...,CHEMBL5370546,1,Scientific Literature,Eur J Med Chem,2023.0,NaN,NaN,INHIBITOR,NaN,0.04
7975,CHEMBL521,IBUPROFEN,4.0,206.28,0.0,3.07,Motrin,CC(C)Cc1ccc(C(C)C(=O)O)cc1,IC50,'=',...,CHEMBL5370523,1,Scientific Literature,Eur J Med Chem,2023.0,NaN,NaN,INHIBITOR,NaN,370.00
7976,CHEMBL139,DICLOFENAC,4.0,296.15,0.0,4.36,DIF,O=C(O)Cc1ccccc1Nc1c(Cl)cccc1Cl,IC50,'=',...,CHEMBL5233249,1,Scientific Literature,Eur J Med Chem,2020.0,NaN,DOSE = 10.0 microM,INHIBITOR,NaN,1.10
7977,CHEMBL121626,TOLFENAMIC ACID,4.0,261.71,0.0,4.09,Tolfenamic acid,Cc1c(Cl)cccc1Nc1ccccc1C(=O)O,IC50,'=',...,CHEMBL5032522,1,Scientific Literature,Eur J Med Chem,2021.0,NaN,TIME = 0.2 hr,INHIBITOR,NaN,0.88


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7979 entries, 0 to 7978
Data columns (total 48 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Molecule ChEMBL ID          7979 non-null   object 
 1   Molecule Name               1782 non-null   object 
 2   Molecule Max Phase          1462 non-null   float64
 3   Molecular Weight            7966 non-null   float64
 4   #RO5 Violations             7873 non-null   float64
 5   AlogP                       7873 non-null   float64
 6   Compound Key                7979 non-null   object 
 7   Smiles                      7945 non-null   object 
 8   Standard Type               7979 non-null   object 
 9   Standard Relation           6978 non-null   object 
 10  Standard Value              6978 non-null   float64
 11  Standard Units              6997 non-null   object 
 12  pChEMBL Value               5653 non-null   float64
 13  Data Validity Comment       340 n

In [4]:
with pd.option_context(
        'display.max_columns', None,
        'display.max_rows', None,
        'display.width', 1000
    ):
    print(df.describe(include='all'))

       Molecule ChEMBL ID Molecule Name  Molecule Max Phase  Molecular Weight  #RO5 Violations        AlogP Compound Key                                             Smiles Standard Type Standard Relation  Standard Value Standard Units  pChEMBL Value  Data Validity Comment                                            Comment    Uo Units  Ligand Efficiency BEI  Ligand Efficiency LE  Ligand Efficiency LLE  Ligand Efficiency SEI  Potential Duplicate Assay ChEMBL ID                                  Assay Description Assay Type BAO Format ID              BAO Label Assay Organism Assay Tissue ChEMBL ID Assay Tissue Name Assay Cell Type Assay Subcellular Fraction               Assay Parameters  Assay Variant Accession  Assay Variant Mutation Target ChEMBL ID       Target Name Target Organism     Target Type Document ChEMBL ID    Source ID     Source Description Document Journal  Document Year Cell ChEMBL ID     Properties Action Type  Standard Text Value          Value
count                7979 

In [5]:
# Проверка, что данные имеют нужный тип активности и содержат численные значения активности в колонке `standard_value`
print(df['Action Type'].value_counts(dropna=False))
print(f"\n{df['Standard Type'].value_counts(dropna=False)}\n\n\n")

df['Standard Value'] = pd.to_numeric(df['Standard Value'], errors='coerce')
df['Standard Value'].info()

Action Type
NaN          7178
INHIBITOR     801
Name: count, dtype: int64

Standard Type
IC50    7979
Name: count, dtype: int64



<class 'pandas.core.series.Series'>
RangeIndex: 7979 entries, 0 to 7978
Series name: Standard Value
Non-Null Count  Dtype  
--------------  -----  
6978 non-null   float64
dtypes: float64(1)
memory usage: 62.5 KB


# Отбор релевантных признаков, обработка пропущенных значений и дубликатов 

- Очистка от признаков с нулевой дисперсией или одним уникальным значением;
- Очистка от признаков, не связанных с биологической активностью или имеющих большое количество пропусков;
- Удаление дублирующих признаков;
- Фильтрация данных на основе эвристик

In [6]:
# Compound Key

# не связанны с биологической активностью 
clean_df = df.drop(df.columns[20:44], axis=1)
clean_df = clean_df.drop(['Molecule Name', 'Molecule Max Phase', 'Standard Type', 'Uo Units', 'Action Type', 'Standard Text Value'], axis=1)

# Properties ?

clean_df

,Molecule ChEMBL ID,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Relation,Standard Value,Standard Units,pChEMBL Value,Data Validity Comment,Comment,Ligand Efficiency BEI,Ligand Efficiency LE,Ligand Efficiency LLE,Ligand Efficiency SEI,Properties,Value
0,CHEMBL70104,364.46,0.0,3.49,3,CC1(C)OC(=O)C(OC2CCCCC2)=C1c1ccc(S(C)(=O)=O)cc1,'=',40.0,nM,7.40,NaN,NaN,20.30,0.40,3.91,10.62,NaN,0.04
1,CHEMBL96098,372.88,0.0,4.43,34,CCc1ccc(-c2ncc(Cl)cc2-c2ccc(S(C)(=O)=O)cc2)cn1,'=',1700.0,nM,5.77,NaN,NaN,15.47,0.32,1.34,9.63,NaN,1.70
2,CHEMBL97179,413.90,1.0,5.19,5,CCCCOC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc(OC)...,'=',50.0,nM,7.30,NaN,NaN,17.64,0.34,2.11,12.69,NaN,0.05
3,CHEMBL6,357.79,0.0,3.93,Indomethacin,COc1ccc2c(c1)c(CC(=O)O)c(C)n2C(=O)c1ccc(Cl)cc1,'<',200.0,nM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.00
4,CHEMBL355781,390.46,0.0,4.25,21,CS(=O)(=O)c1ccc(-c2csc(CC(=O)O)c2-c2ccc(F)cc2)cc1,'>',10000.0,nM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7974,CHEMBL118,381.38,0.0,3.51,Celecoxib,Cc1ccc(-c2cc(C(F)(F)F)nn2-c2ccc(S(N)(=O)=O)cc2...,'=',40.0,nM,7.40,NaN,NaN,19.40,0.39,3.89,9.49,NaN,0.04
7975,CHEMBL521,206.28,0.0,3.07,Motrin,CC(C)Cc1ccc(C(C)C(=O)O)cc1,'=',370000.0,nM,NaN,Outside typical range,NaN,NaN,NaN,NaN,NaN,NaN,370.00
7976,CHEMBL139,296.15,0.0,4.36,DIF,O=C(O)Cc1ccccc1Nc1c(Cl)cccc1Cl,'=',1100.0,nM,5.96,NaN,NaN,20.12,0.43,1.60,12.08,DOSE = 10.0 microM,1.10
7977,CHEMBL121626,261.71,0.0,4.09,Tolfenamic acid,Cc1c(Cl)cccc1Nc1ccccc1C(=O)O,'=',880.0,nM,6.06,NaN,NaN,23.14,0.46,1.97,12.28,TIME = 0.2 hr,0.88


In [7]:
# Маска по Standard Units == 'nM'
units_mask = (
    clean_df['Standard Units']
        .astype(str)        # на случай NaN
        .str.strip()        # убираем внешние пробелы
        .eq('nM')           # сравниваем строго с 'nM'
)

filtered_units_df = clean_df[units_mask].reset_index(drop=True)

# 2. Проверим результат
print("После фильтрации по nM:", filtered_units_df.shape[0], "записей из", len(clean_df))

# 3. Если нужно сразу объединить со всеми предыдущими масками:
# combined_mask = relation_mask & ~comment_mask & ~dv_mask & units_mask
# cleaned_df = clean_df[combined_mask].reset_index(drop=True)


После фильтрации по nM: 6970 записей из 7979


In [8]:
print(clean_df['Standard Relation'].value_counts())


# Маска по точным измерениям
desired_relations = ["'='"]
relation_mask = (
    clean_df['Standard Relation']
    	.astype(str)
        .str.strip()
        .isin(desired_relations)
)

Standard Relation
'='     5948
'>'      973
'<'       56
'>='       1
Name: count, dtype: int64


Standard Relation — текстовый признак, который показывает, как именно интерпретировать численное значение из столбца Standard Value. Другими словами, он указывает, является ли это:

- точным измерением,  
- лишь нижней границей (правостороннее цензурирование),  
- лишь верхней границей (левостороннее цензурирование),  
- либо приблизительным значением.

---

Зачем это нужно

- при регрессии по pIC₅₀
  • оставить только `=` (точные измерения) или использовать модели цензурированной регрессии (Tobit и т. п.)
- при классификации «активен/неактивен»
  • перевести все `>` (IC₅₀ > порог) и/или `<` в класс “inactive”
- при очистке датасета
  • дропать все строки, где Relation ≠ `=` (чтобы не вводить шум от неравноправных меток)

In [9]:
with pd.option_context(
        'display.max_columns', None,
        'display.max_rows', None,
        'display.width', 1000
    ):
    print(clean_df['Comment'].value_counts())


# Маска по комментариям
comment_keywords = ['not active', 'not determined', 'not evaluated', 'inactive']
comment_pattern = '|'.join(map(re.escape, comment_keywords))

comment_mask = (
    clean_df['Comment']
    	.astype(str)
        .str.lower()
        .str.contains(comment_pattern, regex=True, na=False)
)

Comment
Not Active (inhibition < 50% @ 10 uM and thus dose-reponse curve not measured)    841
Not Determined                                                                    111
Not Active                                                                         33
Not Evaluated                                                                      11
Inactive                                                                            3
412267                                                                              1
384890                                                                              1
412272                                                                              1
412284                                                                              1
335051                                                                              1
335052                                                                              1
335064                                        

Вот как логика фильтрации по `Comment` связана с `Standard Relation`:

---

🔍 Почему мы исключаем записи с `Comment` типа "Not Active" и подобные

Фразы вроде:
- `not active`
- `not determined`
- `not evaluated`
- `inactive`

означают, что:
- **соединение не проявило активности** при тестировании (например, <50% ингибирования @ 10 µM);
- **доза-ответ не была измерена**, т.е. точной IC₅₀ нет;
- **эксперимент не дал результатов** или был прерван.

⚠️ Такие записи **не имеют достоверного численного значения активности**, а значит:

- **Standard Value** — это скорее порог, не точное значение;
- **Standard Relation** — обычно _цензурированные данные_.

---

🧠 Связь с фильтрацией по `Standard Relation`

Когда мы отбрасываем строки с `Relation ≠ '='`, мы стремимся оставить **только те записи**, где:

- IC₅₀ (или другой биомаркер) **измерен напрямую**,  
- нет двойных смыслов, ограничений, порогов и приблизительных значений.

Аналогично, если `Comment` намекает, что данные **неактивны / не измерены / неполные**, они **вносят шум** и **искажают регрессию**, особенно по логарифму IC₅₀.

---

💡 Общее правило

Ты фильтруешь:
- по `Standard Relation == '='` — чтобы оставить только **численно точные** активности;
- по `Comment`, исключая неинформативные/недостоверные записи — даже если Relation случайно `=` (но comment намекает на цензуру).

Это **усиливает качество датасета**: в машинном обучении лучше меньше, но точнее.

In [10]:
print(clean_df['Data Validity Comment'].value_counts())


# Маска по ошибочным записям в Data Validity Comment
DV_comment_keywords = ['error', 'outside typical range']
DV_comment_pattern = '|'.join(map(re.escape, DV_comment_keywords))

DV_comment_mask = (
    clean_df['Data Validity Comment']
    	.astype(str)
        .str.lower()
        .str.contains(DV_comment_pattern, regex=True, na=False)
)

Data Validity Comment
Outside typical range            331
Potential transcription error      9
Name: count, dtype: int64


🔍 Что такое `Data Validity Comment`?

Это служебное поле, указывающее на **технические или методологические проблемы с измерением**.  
Примеры:  
- `'Error'` — означает сбой или невозможность корректного измерения  
- `'Outside typical range'` — результат вне доверительного интервала или необычный по методике

---

❌ Почему мы исключаем такие записи

Аналогично предыдущим фильтрам:
- **По `Standard Relation ≠ '='`** — мы отбрасываем **цензурированные или нечеткие** значения  
- **По `Comment`** — исключаем **неактивные** соединения, где дозу даже не определяли  
- **По `Data Validity Comment`** — исключаем **ошибочные или нерепрезентативные** данные  

Все они **вносят шум в регрессионную модель** и мешают корректному обучению.

---

🧠 Почему это особенно важно

Если оставить записи с техническими ошибками:
- может быть записано некорректное IC₅₀  
- дескрипторы для этих соединений не связаны с реальной биологической активностью  
- модель будет учиться на «плохих» или случайных данных

По сути — ты проводишь **тройную очистку**:
1. Только точные численные значения (`Standard Relation == '='`)  
2. Только биологически активные или оценённые соединения (`Comment` не содержит "not active")  
3. Только валидные записи без ошибок (`Data Validity Comment` не содержит 'error' и т. д.)

## Приведение значений активности к единому формату

In [11]:
print("\n", clean_df['Standard Units'].value_counts(dropna=False))


 Standard Units
nM         6970
NaN         982
ug.mL-1      25
ug            1
%             1
Name: count, dtype: int64


In [12]:
# видим, что в колонке `Standard Units` есть разные единицы измерения, следовательно можем написать функцию для конвертации из µg/mL в nM, зная молекулярную массу соединения.

# Функция конверсии из µg/mL → nM
def ug_per_mL_to_nM(ug_per_mL, mw_g_per_mol):
    """
    Переводит концентрацию из µg/mL в nM,
    используя молекулярную массу mw (г/моль).
    """
    # µg/mL → g/L
    g_per_L = ug_per_mL * 1e-6 * 1e3  
    # g/L → mol/L
    mol_per_L = g_per_L / mw_g_per_mol  
    # mol/L → nM
    return mol_per_L * 1e9


# --- Шаг 1: подготовка масок ---

# нормализуем единицы измерения (например, 'µg/mL', 'ug/ml', 'ug.mL-1')
standard_units = clean_df['Standard Units'].astype(str).str.lower().str.strip()

# строки, где единицы равны 'nM'
mask_nM = standard_units == 'nm'

# строки с µg/mL — потенциально приводимые
mask_ug_mL = standard_units.isin(['ug.ml-1', 'µg/ml', 'ug/ml'])


# --- Шаг 2: создаём новый столбец для концентрации ---
clean_df['Concentration_nM'] = np.nan

# Копируем значения из Standard Value, если они уже в nM
clean_df.loc[mask_nM, 'Concentration_nM'] = clean_df.loc[mask_nM, 'Standard Value']

# Конвертируем значения из µg/mL в nM, если присутствуют и MW, и Standard Value
clean_df.loc[mask_ug_mL, 'Concentration_nM'] = clean_df.loc[mask_ug_mL].apply(
    lambda row: ug_per_mL_to_nM(row['Standard Value'], row['Molecular Weight'])
    if pd.notnull(row['Standard Value']) and pd.notnull(row['Molecular Weight']) and row['Molecular Weight'] > 0
    else np.nan,
    axis=1
)

# заменяем единицы на 'nM' для сконвертированных строк ---
converted_mask = mask_ug_mL & clean_df['Concentration_nM'].notna()
clean_df.loc[converted_mask, 'Standard Units'] = 'nM'

# маска
unit_mask = clean_df['Standard Units'].astype(str).str.lower().str.strip().eq('nm')

In [13]:
# Фильтрация первично очищенных данных по комбинированной маске и удаление полных дублей

combined_mask = relation_mask & ~comment_mask & ~DV_comment_mask & unit_mask
filtered_df = clean_df[combined_mask].copy()


# смотрим, сколько полных дубликатов есть
orig_len = len(filtered_df)
dupe_mask = filtered_df.duplicated(keep=False)   # True для всех строк, которые имеют дубликаты
n_dupes = dupe_mask.sum()
print(f"Найдены {n_dupes} строк(а) с полными дубликатами из {orig_len} после фильтрации.")

# удаляем полные дубликаты (все столбцы) и сбрасываем индекс
f_deduped_df = (
    filtered_df
      	.drop_duplicates(keep='first')   # оставляет первую встреченную строку из каждой группы дублей
    	.reset_index(drop=True)
)
new_len = len(f_deduped_df)
print(f"После удаления дублей осталось {new_len} строк.\n\n\n")


# валидация после фильтрации
print(f'{f_deduped_df['Standard Relation'].value_counts(dropna=False)}\n\n\n')
print(f'{f_deduped_df['Comment'].value_counts(dropna=False)}\n\n\n')
print(f'{f_deduped_df['Data Validity Comment'].value_counts(dropna=False)}\n\n\n')


print(f'{f_deduped_df['Standard Units'].value_counts(dropna=False)}\n\n\n')

print(f'{f_deduped_df['Concentration_nM'].describe()}')

Найдены 223 строк(а) с полными дубликатами из 5666 после фильтрации.
После удаления дублей осталось 5530 строк.



Standard Relation
'='    5530
Name: count, dtype: int64



Comment
NaN       5477
335051       1
335052       1
335064       1
335085       1
335077       1
335081       1
335083       1
335092       1
335054       1
335074       1
335076       1
412290       1
335056       1
335080       1
335086       1
335071       1
335063       1
335082       1
335088       1
335057       1
335059       1
335046       1
335069       1
335084       1
335079       1
335089       1
335050       1
335070       1
335073       1
335049       1
335061       1
335094       1
335062       1
335087       1
335055       1
335072       1
335078       1
335068       1
335095       1
335053       1
335075       1
335047       1
335066       1
335090       1
384888       1
335048       1
335058       1
335060       1
335065       1
335091       1
335067       1
384889       1
335093       1
Name: co

In [14]:
with pd.option_context(
        'display.max_columns', None,
        'display.max_rows', None,
        'display.width', 1000
    ):
    print(f_deduped_df.describe(include='all'))

       Molecule ChEMBL ID  Molecular Weight  #RO5 Violations        AlogP Compound Key                                             Smiles Standard Relation  Standard Value Standard Units  pChEMBL Value Data Validity Comment Comment  Ligand Efficiency BEI  Ligand Efficiency LE  Ligand Efficiency LLE  Ligand Efficiency SEI     Properties         Value  Concentration_nM
count                5530       5529.000000      5499.000000  5499.000000         5530                                               5529              5530     5530.000000           5530    5517.000000                     0      53            5427.000000           5397.000000            5397.000000            5395.000000            450   5530.000000       5530.000000
unique               4076               NaN              NaN          NaN         1760                                               4075                 1             NaN              1            NaN                     0      53                    NaN      

In [15]:
# 1) Задаём колонки для агрегации
bioactivity_cols = ['pChEMBL Value', 'Standard Value', 'Concentration_nM']
physicochemical_cols = [
    'Molecular Weight',
    'AlogP',
    '#RO5 Violations',
    'Ligand Efficiency BEI',
    'Ligand Efficiency LE',
    'Ligand Efficiency LLE',
    'Ligand Efficiency SEI'
]
id_col = 'Molecule ChEMBL ID'

# 2) Составляем словарь агрегаторов
agg_dict = {}

for col in bioactivity_cols:
    if col in f_deduped_df.columns:
        agg_dict[col] = 'median'

for col in physicochemical_cols:
    if col in f_deduped_df.columns:
        agg_dict[col] = 'mean'

if id_col in f_deduped_df.columns:
    agg_dict[id_col] = lambda x: ','.join(sorted(set(x.dropna().astype(str))))

# 3) Группируем по Smiles
aggregated_df = (
    f_deduped_df
      .groupby('Smiles', as_index=False)
      .agg(agg_dict)
)

# 4) Рассчитываем pIC50 из медианной Concentration_nM
if 'Concentration_nM' in aggregated_df.columns:
    aggregated_df['pIC50'] = -np.log10(aggregated_df['Concentration_nM'] * 1e-9)

# 5) Дропим колонки без предиктивной ценности
to_drop = ['Standard Units', 'Standard Relation', 'Compound Key']
aggregated_df = aggregated_df.drop(columns=to_drop, errors='ignore')

# 6) Удаляем все оставшиеся NaN и сбрасываем индекс
aggregated_df = aggregated_df.dropna().reset_index(drop=True)

# 7) Итоговый просмотр
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.width', 1200):
    print(aggregated_df)


                                                 Smiles  pChEMBL Value  Standard Value  Concentration_nM  Molecular Weight  AlogP  #RO5 Violations  Ligand Efficiency BEI  Ligand Efficiency LE  Ligand Efficiency LLE  Ligand Efficiency SEI Molecule ChEMBL ID      pIC50
0          Br.CSc1nc2ccccc2n1-c1csc(=N)n1CC(=O)c1ccccc1          6.750      180.000000        180.000000            461.41   3.97              0.0              17.730000              0.350000               2.770000              10.590000      CHEMBL4637009   6.744727
1     Brc1ccc(-c2nn(-c3ccccc3)cc2-c2nnc(-c3ccncc3)o2...          4.410    39000.000000      39000.000000            444.29   5.41              1.0               9.920000              0.210000              -1.000000               6.330000      CHEMBL3265356   4.408935
2            Brc1ccc(-c2nnc(CSc3nnc(-c4ccncc4)o3)o2)cc1          7.280       53.000000         53.000000            416.26   4.24              0.0              17.480000              0.400000     

In [16]:
with pd.option_context(
        'display.max_columns', None,
        'display.max_rows', None,
        'display.width', 1000
    ):
    print(aggregated_df.describe(include='all'))

                                              Smiles  pChEMBL Value  Standard Value  Concentration_nM  Molecular Weight        AlogP  #RO5 Violations  Ligand Efficiency BEI  Ligand Efficiency LE  Ligand Efficiency LLE  Ligand Efficiency SEI Molecule ChEMBL ID        pIC50
count                                           3967    3967.000000     3967.000000       3967.000000       3967.000000  3967.000000      3967.000000            3967.000000           3967.000000            3967.000000            3967.000000               3967  3967.000000
unique                                          3967            NaN             NaN               NaN               NaN          NaN              NaN                    NaN                   NaN                    NaN                    NaN               3967          NaN
top     Br.CSc1nc2ccccc2n1-c1csc(=N)n1CC(=O)c1ccccc1            NaN             NaN               NaN               NaN          NaN              NaN                    NaN         

In [17]:
# Проверка на невалидные smiles путём создания mol файла, индексы некорректных smiles сохраняются для дальнейшего удаления
def valid_smiles(df, smiles_column='Smiles'):
    invalid_indices = []
    
    for idx, smiles in df[smiles_column].items():
        try:
            mol = Chem.MolFromSmiles(str(smiles))
            if mol is None:
                invalid_indices.append(idx)
        except:
            invalid_indices.append(idx)
    
    return invalid_indices

invalid_indices = valid_smiles(aggregated_df, 'Smiles')
print(invalid_indices)
# pure_df = aggregated_df.drop(index=invalid_indices) 

[]


In [18]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.width', 1200):
    print(aggregated_df)

                                                 Smiles  pChEMBL Value  Standard Value  Concentration_nM  Molecular Weight  AlogP  #RO5 Violations  Ligand Efficiency BEI  Ligand Efficiency LE  Ligand Efficiency LLE  Ligand Efficiency SEI Molecule ChEMBL ID      pIC50
0          Br.CSc1nc2ccccc2n1-c1csc(=N)n1CC(=O)c1ccccc1          6.750      180.000000        180.000000            461.41   3.97              0.0              17.730000              0.350000               2.770000              10.590000      CHEMBL4637009   6.744727
1     Brc1ccc(-c2nn(-c3ccccc3)cc2-c2nnc(-c3ccncc3)o2...          4.410    39000.000000      39000.000000            444.29   5.41              1.0               9.920000              0.210000              -1.000000               6.330000      CHEMBL3265356   4.408935
2            Brc1ccc(-c2nnc(CSc3nnc(-c4ccncc4)o3)o2)cc1          7.280       53.000000         53.000000            416.26   4.24              0.0              17.480000              0.400000     

In [22]:
# 1.1. Считаем количество и долю пропусков в каждом столбце
missing_counts   = aggregated_df.isna().sum()
missing_percent  = aggregated_df.isna().mean() * 100

# 1.2. Формируем удобную табличку  
missing_summary = pd.DataFrame({
    'missing_count':   missing_counts,
    'missing_percent': missing_percent
}).sort_values(by='missing_count', ascending=False)

# 1.3. Показываем результат
print("Пропущенные значения по столбцам:\n")
print(missing_summary)


Пропущенные значения по столбцам:

                       missing_count  missing_percent
Smiles                             0              0.0
pChEMBL Value                      0              0.0
Standard Value                     0              0.0
Concentration_nM                   0              0.0
Molecular Weight                   0              0.0
AlogP                              0              0.0
#RO5 Violations                    0              0.0
Ligand Efficiency BEI              0              0.0
Ligand Efficiency LE               0              0.0
Ligand Efficiency LLE              0              0.0
Ligand Efficiency SEI              0              0.0
Molecule ChEMBL ID                 0              0.0
pIC50                              0              0.0


In [23]:
output_path = '../../data/target_cox2_IC50__preprocessed_aggregated.csv'
aggregated_df.to_csv(output_path, index=False)
print(f"\nИтоговый датасет сохранён в: {output_path}")



Итоговый датасет сохранён в: ../../data/target_cox2_IC50__preprocessed_aggregated.csv


**Почему для разных признаков выбраны среднее (mean) и медиана (median)**

---

1. Медиана для биоактивностей  
- **Признаки**:  
  - pChEMBL Value  
  - Standard Value (обычно IC₅₀/EC₅₀ и т. д.)  
  - Concentration_nM  

**Причина:**  
1. Биоактивность — это часто сильно скошенные данные (несколько очень высоких или очень низких значений).  
2. Медиана устойчива к выбросам: если в эксперименте одна точка «вышла из рядов», она не раздувает агрегат.  
3. Мы работаем с логарифмическими шкалами (pChEMBL, pIC₅₀), где важнее «типичное» значение, а не среднее арифметическое.  

Пример  
- IC₅₀ = [1 nM, 2 nM, 1000 nM]  
  - Среднее = 334 nM (не отражает, где лежат «обычные» точки)  
  - Медиана = 2 nM (ближе к реальной средней активности комплекта)

---

2. Среднее для физико-химических признаков  
- **Признаки**:  
  - Molecular Weight  
  - AlogP  
  - #RO5 Violations  
  - Ligand Efficiency BEI, LE, LLE, SEI  

**Причина:**  
1. Эти свойства часто распределены примерно нормально вокруг среднего (особенно в «библиотеках лидов»).  
2. Среднее отражает «центральное» значение характеристики, важное для дизайн-решений.  
3. Поскольку выбросы здесь редки и обычно указывают на аномальные структуры, их влияние несущественно для общей картины.  

Пример  
- AlogP = [1.2, 1.5, 1.7, 8.0]  
  - Среднее ≈ 3.6 (показывает «среднюю» липофильность партии)  
  - Медиана = 1.6 (скрывает несколько сильно липофильных аутсайдеров)

---

3. Итог  
- **Медиана** для сконцентрированных, скошенных показателей активности — она защищает от единичных «залипаний» и даёт репрезентативную картину биологического ответа.  
- **Среднее** для свойства молекул, где распределение более симметрично, и важно знать именно арифметический центр данных.  

Такой подход помогает строить модели и делать выводы на стабильной основе:  
- активности агрегируются по «типичному» ответу,  
- физико-химия проверяется на общий баланс свойств без чрезмерного влияния аутсайдеров.